# AP1roG calculations

In [1]:
# Force the local gqcpy to be imported
import sys
sys.path.insert(0, '../../build/gqcpy/')

import gqcpy
import numpy as np

np.set_printoptions(precision=3, linewidth=120)

## Setting up the molecular Hamiltonian in the canonical RHF spin-orbital basis

The AP1roG geminal pairing scheme is set up in the canonical RHF (spin-orbital) basis, so we'll have to do an RHF calculation first. There are plenty of other examples that focus on Hartree-Fock calculations in more detail.

In [2]:
molecule = gqcpy.Molecule.ReadXYZ("../../gqcp/tests/data/ch4_crawdad.xyz", 0)  # '0': Create a neutral molecule.
N = molecule.numberOfElectrons()
N_P = molecule.numberOfElectronPairs()

In [3]:
spin_orbital_basis = gqcpy.RSpinOrbitalBasis(molecule, "STO-3G")  # The basis of the atomic spin-orbitals.
K = spin_orbital_basis.numberOfSpatialOrbitals()

S = spin_orbital_basis.quantizeOverlapOperator().parameters()

Since the initial spin-orbital basis is the AO basis, the Hamiltonian is expressed in the non-orthogonal AOs. This is exactly what we need to start an RHF calculation.

In [4]:
hamiltonian = gqcpy.RSQHamiltonian.Molecular(spin_orbital_basis, molecule)

In [5]:
environment = gqcpy.RHFSCFEnvironment.WithCoreGuess(N, hamiltonian, S)
solver = gqcpy.RHFSCFSolver.DIIS()

objective = gqcpy.DiagonalRHFFockMatrixObjective(hamiltonian)

Using this objective makes sure that the optimized expansion coefficients yield a diagonal Fock matrix, so the solution represents the canonical RHF spinor basis.

In [6]:
rhf_parameters = gqcpy.RHF.optimize(objective, solver, environment).groundStateParameters()
C = rhf_parameters.expansion()

Since we have the canonical RHF spinor expansion coefficients now, we can transform the underlying spin-orbital basis and then re-quantize the molecular Hamiltonian, in order to let both instances be in-sync with their basis transformations.

However, gqcpy offers a different approach, transforming the spin-orbital basis and the Hamiltonian with their member APIs

In [7]:
spin_orbital_basis.transform(C)
hamiltonian.transform(C)

Right now, the spin-orbital basis and Hamiltonian are expressed in the canonical RHF spin-orbitals.

## AP1roG calculations

AP1roG has been formulated as a QCModel/QCMethod. Therefore, we should create a solver (which is able to solve the AP1roG PSEs), an associated environment and an objective. For AP1roG, the solver is a non-linear equation solver.

In [8]:
environment = gqcpy.PSEnvironment.AP1roG(hamiltonian, N_P)
solver = gqcpy.NonLinearEquationSolver.Newton()

In [9]:
qc_structure = gqcpy.AP1roG(hamiltonian, N_P).optimize(solver, environment)

In [10]:
print(qc_structure.groundStateEnergy())

-53.25121300703414


In [11]:
print(qc_structure.groundStateParameters().geminalCoefficients().asMatrix())

[[ 1.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00 -7.789e-04 -7.792e-04 -7.785e-04 -1.948e-03]
 [ 0.000e+00  1.000e+00  0.000e+00  0.000e+00  0.000e+00 -1.414e-02 -1.420e-02 -1.408e-02 -2.450e-02]
 [ 0.000e+00  0.000e+00  1.000e+00  0.000e+00  0.000e+00 -4.253e-02 -3.813e-02 -9.759e-03 -1.866e-02]
 [ 0.000e+00  0.000e+00  0.000e+00  1.000e+00  0.000e+00 -4.025e-02 -3.044e-02 -1.881e-02 -1.873e-02]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  1.000e+00 -7.701e-03 -2.091e-02 -6.298e-02 -1.856e-02]]


## vAP1roG calculations

We have also extended the AP1roG method to be variationally optimized, resulting in the vAP1roG method. In short, what this method does is analogous to AP1roG, but after determining the optimal geminal coefficients, a set of optimal Lagrange multipliers is also searched for.

Since these Lagrange multipliers are determined through solving a linear equation, we will have to supply a linear equations solver to the vAP1roG method.

In [12]:
non_linear_environment = gqcpy.PSEnvironment.AP1roG(hamiltonian, N_P)
non_linear_solver = gqcpy.NonLinearEquationSolver.Newton()

linear_solver = gqcpy.LinearEquationSolver.ColPivHouseholderQR()

In [13]:
qc_structure = gqcpy.vAP1roG(hamiltonian, N_P).optimize(non_linear_solver, non_linear_environment, linear_solver)

In [14]:
print(qc_structure.groundStateEnergy())

-53.25121300703414


In [15]:
print(qc_structure.groundStateParameters().geminalCoefficients().asMatrix())

[[ 1.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00 -7.789e-04 -7.792e-04 -7.785e-04 -1.948e-03]
 [ 0.000e+00  1.000e+00  0.000e+00  0.000e+00  0.000e+00 -1.414e-02 -1.420e-02 -1.408e-02 -2.450e-02]
 [ 0.000e+00  0.000e+00  1.000e+00  0.000e+00  0.000e+00 -4.253e-02 -3.813e-02 -9.759e-03 -1.866e-02]
 [ 0.000e+00  0.000e+00  0.000e+00  1.000e+00  0.000e+00 -4.025e-02 -3.044e-02 -1.881e-02 -1.873e-02]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  1.000e+00 -7.701e-03 -2.091e-02 -6.298e-02 -1.856e-02]]


In [16]:
print(qc_structure.groundStateParameters().lagrangeMultipliers())

[[-0.001 -0.001 -0.001 -0.002]
 [-0.014 -0.014 -0.014 -0.025]
 [-0.042 -0.038 -0.01  -0.019]
 [-0.04  -0.03  -0.019 -0.019]
 [-0.008 -0.021 -0.063 -0.019]]


## Orbital optimization for vAP1roG

We've also implemented an second-order orbital optimizer that uses a Newton step in every iteration. (The API isn't quite up-to-par yet.)

The orbital optimize requires an initial guess.

In [17]:
G_initial = qc_structure.groundStateParameters().geminalCoefficients()

optimizer = gqcpy.AP1roGLagrangianNewtonOrbitalOptimizer(G_initial, oo_convergence_threshold=1.0e-04)
optimizer.optimize(spin_orbital_basis, hamiltonian)

We can see that the electronic energy has lowered due to the orbital optimization.

In [18]:
print(optimizer.electronicEnergy())

-53.28557074515322


The converged geminal coefficients and multipliers can be found, too.

In [19]:
print(optimizer.geminalCoefficients().asMatrix())

[[ 1.     0.     0.     0.     0.    -0.001 -0.001 -0.001 -0.001]
 [ 0.     1.     0.     0.     0.    -0.007 -0.007 -0.007 -0.091]
 [ 0.     0.     1.     0.     0.    -0.007 -0.091 -0.007 -0.007]
 [ 0.     0.     0.     1.     0.    -0.091 -0.007 -0.007 -0.007]
 [ 0.     0.     0.     0.     1.    -0.007 -0.007 -0.091 -0.007]]


In [20]:
print(optimizer.multipliers())

[[-0.001 -0.001 -0.001 -0.001]
 [-0.007 -0.007 -0.007 -0.091]
 [-0.007 -0.091 -0.007 -0.007]
 [-0.091 -0.007 -0.007 -0.007]
 [-0.007 -0.007 -0.091 -0.007]]
